In [1]:
! python -m pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 110.2 MB/s eta 0:00:00


In [8]:
import json
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Lettura file CSV
df = pd.read_csv("dieta2.csv")
"""
imposto la prima colonna ('i' in questo caso) come indice di riga per poter accedere ad una riga del file utilizzando .loc
riga di codice equivalente per ottenere lo stesso risultato: df = df.set_index('i')
"""
idx = df.columns.tolist()[0]
df = df.set_index(f'{idx}')

# Definizione degli insiemi
"""
Seleziono la riga con indice 'costo' e la converto in un dizionario:
- chiavi = cibi (colonne del DataFrame)
- valori = costo unitario di ciascun cibo

C = { 'pane': 2.0,
      'latte': 1.5,
      'uova': 0.8,
      ... }
"""
C = df.loc["costo"].to_dict()


"""
Seleziono la colonna 'domanda' e la converto in un dizionario:
- chiavi = nutrienti (righe del DataFrame)
- valori = quantità minima richiesta per ciascun nutriente

D = { 'proteine': 50,
      'grassi': 20,
      'vitamine': 10,
      ... }
"""
D = df["domanda"].to_dict()

"""
Creo la matrice A eliminando le righe e le colonne 'costo' e 'domanda'
- la riga 'costo' è eliminata perché non è un nutriente
- la colonna 'domanda' è eliminata perché non è un cibo

A[j,i] indica quanta quantità del nutriente j fornisce una unità del cibo i.
NB: le righe = nutrienti, le colonne = cibi.
"""
A  = df.drop(index="costo").drop(columns="domanda")

"""
Ottengo l’insieme I (cibi disponibili):
lista dei nomi delle colonne della matrice A.
I = ['pane', 'latte', 'uova', ...]
"""
I = A.columns.tolist()

"""
Ottengo l’insieme J (nutrienti):
lista dei nomi delle righe della matrice A.
J = ['proteine', 'grassi', 'vitamine', ...]
"""
J = A.index.tolist()


print("Costi (C):", C)
print("Domande (D):", D)
print("Matrice (A):\n", A)
print("Cibi (I):", I)
print("Nutrienti (J):", J)


# Creazione modello
m = gp.Model("dieta2")
m.setParam("OutputFlag", 0)

# Creazione variabili decisionali
x = m.addVars(I, vtype=GRB.INTEGER, lb=0, name="x")

# Creazione vincoli
for j in J:
    sum = gp.quicksum(A.loc[j, i] * x[i] for i in I)
    m.addConstr(
        sum >= D[j],
        name=f"nutriente_{j}",
    )

# Nuovo vincolo: almeno 3kg di alimenti a base di pesce azzurro
m.addConstr(x['pesce'] >= 3)

# Definizione o.f.
m.setObjective(gp.quicksum(C[i] * x[i] for i in I), GRB.MINIMIZE)

# Ottimizzazione
m.optimize()

# Visualizzazione risultati
if m.status == GRB.OPTIMAL:
    print(f"O.f. (costo) = {m.objVal:.3f}\n")
    for i in I:
        print(f"x[{i}] = {int(x[i].X):.2f}")

Costi (C): {'verdura': 4.0, 'carne': 10.0, 'frutta': 7.0, 'pesce': 3.0, 'domanda': nan}
Domande (D): {'proteine': 20.0, 'ferro': 30.0, 'calcio': 10.0, 'costo': nan}
Matrice (A):
           verdura  carne  frutta  pesce
i                                      
proteine        5     15       4     10
ferro           6     10       5     15
calcio          5      3      12      2
Cibi (I): ['verdura', 'carne', 'frutta', 'pesce']
Nutrienti (J): ['proteine', 'ferro', 'calcio']
O.f. (costo) = 13.000

x[verdura] = 1.00
x[carne] = 0.00
x[frutta] = 0.00
x[pesce] = 3.00
